In [58]:
import pandas as pd
import os
import re
import ast
from autolabel import AutolabelDataset
from few_shot_new import NewAutoLabelConfig
from sklearn.metrics import classification_report
from autolabel.utils import print_table

In [62]:
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.test.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "meta-llama/Llama-2-13b-chat-hf"
label_column = 'sentence_class'
text_column = 'tweet'
explanation_column = 'sentence_explanation'
few_shot_selection = "semantic_similarity"
example_selection_label_column = 'sentence_class'
use_current_explanation = False
use_ground_explanation = True

In [63]:
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [64]:
predictions = pd.read_csv(output_path)

In [35]:
def process_response(annotation):
    response = annotation.split("raw_response=")[1]
    if response.find('Output: [/INST]\\n') < response.find('explanation='):
        response = response.split("Output: [/INST]\\n")[-1].split("explanation=")[0]
    else:
        response = response.split("explanation=")[0]
    return response

def clean_text(text):
    # Remove leading and trailing single quotes, spaces, backslashes, and newlines
    cleaned = re.sub(r"^['\\s\\n]+|['\\s\\n]+$", '', text)
    return cleaned

def clean_text_refined(s):
    # First, strip the leading and trailing whitespace (which includes newlines)
    s = s.strip()

    # Then remove a leading or trailing single quote if present
    if s.startswith("'"):
        s = s[1:]
    if s.endswith("'"):
        s = s[:-1]

    # Finally, strip again to remove any extra whitespace left after removing quotes
    s = s.strip()
    return s

def get_label(response):
    if 'yes' in response.lower():
        return 'Yes'
    elif 'no' in response.lower():
        return 'No'
    else:
        return 'NO_LABEL'
    
def process_and_get_label(annotation):
    response = process_response(annotation)
    label = get_label(response)
    return label

In [49]:
def get_report(llm_labels, gt_labels):
    filtered_llm_labels = []
    filtered_gt_labels = []
    for llm_label, gt_label in zip(llm_labels, gt_labels):
        if llm_label != "NO_LABEL" and gt_label != "nan":
            filtered_llm_labels.append(llm_label)
            filtered_gt_labels.append(gt_label)
    report = classification_report(filtered_gt_labels, filtered_llm_labels)
    return report

In [16]:
re.sub(r"^['\\s\\n]+|['\\s\\n]+$", '', "'\\nOutput: Location' ")

"Output: Location' "

In [65]:
predictions[f'{label_column}_label'].unique()

array(['Yes', 'No', 'NO_LABEL'], dtype=object)

In [68]:
predictions.loc[predictions[f'{label_column}_error'].notna(), f'{label_column}_label'] = predictions.loc[predictions[f'{label_column}_error'].notna(), f'{label_column}_annotation'].apply(process_and_get_label)

In [69]:
report = get_report(predictions[f'{label_column}_label'], predictions[label_column])
print(report)

              precision    recall  f1-score   support

          No       0.91      0.39      0.55       275
         Yes       0.43      0.93      0.59       137

    accuracy                           0.57       412
   macro avg       0.67      0.66      0.57       412
weighted avg       0.75      0.57      0.56       412



In [131]:
predictions[f'{label_column}_annotation'].apply(process_response).apply(clean_text_refined).unique()

array(['Food poisoning%Symtom', '', '\\nFood Keyword',
       '\\nFood: bel bev dabo', 'food%Food', '\\nFood: Swanson st',
       '\\nInput: @USER just puked at home  #foodpoisoning in Brazil',
       'Food: Cheddars', 'Symtom', '\\nFood poisoning%Symptom',
       '\\nFood poisoning%Keyword', '\\nFood: HTTPURL', 'Food: Fever',
       '\\nFood: Chicken rice', '\\nFood: Middletown NY', '\\nSymtom',
       '\\nFood: #norovirus', '\\nFood:', '\\nOutput: Food:', '\\n',
       '\\nFood handling regulations%Keyword', '\\nFood: cat',
       '\\nSymptom', '\\nFood: food',
       'Food: food  Food poisoning: food poisoning  Symtom: nauseated',
       '\\n\\n', '\\nFood: indigestionFood', 'pizza%Food, noodles%Food',
       '\\nInput: @LYLASMJ alcohol poisoning is a long term disease & dr alcohol should pay for it, find out also where the dr gets his alcohol from',
       '\\nFood: garri', '\\nFood: fish', '\\nFood: food poisoning',
       '\\nFood: sandwich', '\\nFood: gluten', 'Food: pufferfish'

In [ ]:
predictions

In [ ]:
predictions[f'{label_column}_label'][4]